# Week 1: Find Clusters of Infected People

<span style="color:red">
**URGENT WARNING**

We have been receiving reports from health facilities that a new, fast-spreading virus has been discovered in the population. To prepare our response, we need to understand the geospatial distribution of those who have been infected. Find out whether there are identifiable clusters of infected individuals and where they are.    
</span>

Your goal for this notebook will be to estimate the location of dense geographic clusters of infected people using incoming data from week 1 of the simulated epidemic.

## Imports

In [45]:
import cudf
import cuml

import cupy as cp

## Load Data

Begin by loading the data you've received about week 1 of the outbreak into a cuDF data frame. The data is located at `'./data/week1.csv'`. For this notebook you will only need the `'lat'`, `'long'`, and `'infected'` columns. Either drop the columns after loading, or use the `cudf.read_csv` named argument `usecols` to provide a list of only the columns you need.

In [46]:
df = cudf.read_csv("./data/week1.csv", usecols=["lat","long","infected"])
df["lat"] = df["lat"].astype('float32')
df["long"] = df["long"].astype('float32')
print(df.dtypes)
df

lat         float32
long        float32
infected       bool
dtype: object


,lat,long,infected
0,54.522511,-1.571896,False
1,54.554031,-1.524969,False
2,54.552486,-1.435203,False
3,54.537189,-1.566215,False
4,54.528210,-1.588462,False
...,...,...,...
58479889,51.634415,-2.925863,False
58479890,51.556973,-3.036290,False
58479891,51.588993,-2.921915,False
58479892,51.590973,-2.954540,False


## Make Data Frame of the Infected

Make a new cuDF data frame `infected_df` that contains only the infected members of the population.

In [47]:
infected_df = df[df["infected"] == True].reset_index()
infected_df

,index,lat,long,infected
0,28928759,54.472767,-1.654932,True
1,28930512,54.529716,-1.667143,True
2,28930904,54.512985,-1.589866,True
3,28932226,54.522324,-1.380694,True
4,28933748,54.541660,-1.613490,True
...,...,...,...,...
18143,57404109,52.428349,-3.322932,True
18144,57406802,52.415894,-3.263942,True
18145,57410428,52.539932,-3.617128,True
18146,57411005,52.435490,-3.597263,True


## Make Grid Coordinates for Infected Locations

Provided for you in the next cell (which you can expand by clicking on the "..." and contract again after executing by clicking on the blue left border of the cell) is the lat/long to OSGB36 grid coordinates converter you used earlier in the workshop. Use this converter to create grid coordinate values stored in `northing` and `easting` columns of the `infected_df` you created in the last step.

In [48]:
# https://www.ordnancesurvey.co.uk/docs/support/guide-coordinate-systems-great-britain.pdf

def latlong2osgbgrid_cupy(lat, long, input_degrees=True):
    '''
    Converts latitude and longitude (ellipsoidal) coordinates into northing and easting (grid) coordinates, using a Transverse Mercator projection.
    
    Inputs:
    lat: latitude coordinate (N)
    long: longitude coordinate (E)
    input_degrees: if True (default), interprets the coordinates as degrees; otherwise, interprets coordinates as radians
    
    Output:
    (northing, easting)
    '''
    
    if input_degrees:
        lat = lat * cp.pi/180
        long = long * cp.pi/180

    a = 6377563.396
    b = 6356256.909
    e2 = (a**2 - b**2) / a**2

    N0 = -100000 # northing of true origin
    E0 = 400000 # easting of true origin
    F0 = .9996012717 # scale factor on central meridian
    phi0 = 49 * cp.pi / 180 # latitude of true origin
    lambda0 = -2 * cp.pi / 180 # longitude of true origin and central meridian
    
    sinlat = cp.sin(lat)
    coslat = cp.cos(lat)
    tanlat = cp.tan(lat)
    
    latdiff = lat-phi0
    longdiff = long-lambda0

    n = (a-b) / (a+b)
    nu = a * F0 * (1 - e2 * sinlat ** 2) ** -.5
    rho = a * F0 * (1 - e2) * (1 - e2 * sinlat ** 2) ** -1.5
    eta2 = nu / rho - 1
    M = b * F0 * ((1 + n + 5/4 * (n**2 + n**3)) * latdiff - 
                  (3*(n+n**2) + 21/8 * n**3) * cp.sin(latdiff) * cp.cos(lat+phi0) +
                  15/8 * (n**2 + n**3) * cp.sin(2*(latdiff)) * cp.cos(2*(lat+phi0)) - 
                  35/24 * n**3 * cp.sin(3*(latdiff)) * cp.cos(3*(lat+phi0)))
    I = M + N0
    II = nu/2 * sinlat * coslat
    III = nu/24 * sinlat * coslat ** 3 * (5 - tanlat ** 2 + 9 * eta2)
    IIIA = nu/720 * sinlat * coslat ** 5 * (61-58 * tanlat**2 + tanlat**4)
    IV = nu * coslat
    V = nu / 6 * coslat**3 * (nu/rho - cp.tan(lat)**2)
    VI = nu / 120 * coslat ** 5 * (5 - 18 * tanlat**2 + tanlat**4 + 14 * eta2 - 58 * tanlat**2 * eta2)

    northing = I + II * longdiff**2 + III * longdiff**4 + IIIA * longdiff**6
    easting = E0 + IV * longdiff + V * longdiff**3 + VI * longdiff**5

    return(northing, easting)

In [49]:
cupy_lat = cp.asarray(infected_df['lat'])
cupy_long = cp.asarray(infected_df['long'])

northing, easting = latlong2osgbgrid_cupy(cupy_lat,cupy_long)

infected_df['northing'] = northing.astype('float32')
infected_df['easting'] = easting.astype('float32')

infected_df

,index,lat,long,infected,northing,easting
0,28928759,54.472767,-1.654932,True,508670.62500,422359.75000
1,28930512,54.529716,-1.667143,True,515003.06250,421538.53125
2,28930904,54.512985,-1.589866,True,513167.68750,426549.87500
3,28932226,54.522324,-1.380694,True,514305.90625,440081.21875
4,28933748,54.541660,-1.613490,True,516349.56250,425003.00000
...,...,...,...,...,...,...
18143,57404109,52.428349,-3.322932,True,282016.87500,310060.09375
18144,57406802,52.415894,-3.263942,True,280559.56250,314046.12500
18145,57410428,52.539932,-3.617128,True,294832.75000,290338.18750
18146,57411005,52.435490,-3.597263,True,283187.78125,291428.28125


## Find Clusters of Infected People

Use DBSCAN to find clusters of at least 25 infected people where no member is more than 2000m from at least one other cluster member. Create a new column in `infected_df` which contains the cluster to which each infected person belongs.

In [50]:
dbscan = cuml.DBSCAN(eps=2000,min_samples=25)
infected_df['cluster'] = dbscan.fit_predict(infected_df[['northing', 'easting']])
infected_df['cluster'].nunique()

15

## Find the Centroid of Each Cluster

Use grouping to find the mean `northing` and `easting` values for each cluster identified above.

In [51]:
cluster_group = infected_df[['northing', 'easting']].groupby(infected_df['cluster'])
centroids = cluster_group.mean()
centroids

,northing,easting
cluster,,
-1,378086.033377,401876.823293
0,397661.088215,371410.389442
1,436475.647059,332980.470588
2,347062.471464,389387.076923
3,359668.920000,379638.080000
4,391630.393939,431158.090909
5,386471.558140,426559.116279
6,434970.629630,406985.259259
7,412772.974359,410069.717949


Find the number of people in each cluster by counting the number of appearances of each cluster's label in the column produced by DBSCAN.

In [52]:
cluster_group = infected_df["infected"].groupby(infected_df['cluster'])
people = cluster_group.count()
display(people)

cluster
-1     8449
 0     8638
 1       68
 2      403
 3       25
 4       66
 5       43
 6       27
 7       39
 8       94
 9       21
 10      64
 11      68
 12      72
 13      71
Name: infected, dtype: int32

## Take the Assessment

After completing the work above, visit the *Launch Section* web page that you used to launch this Jupyter Lab. Scroll down below where you launched Jupyter Lab, and answer the question *Week 1 Assessment*. You can view your overall progress in the assessment by visiting the same *Launch Section* page and clicking on the link to the *Progress* page.

There will be additional questions for you to answer after completing the remaining notebooks. On the *Progress* page, if you have successfully answered all the assessment questions, you can click on *Generate Certificate* to receive your certificate in the course.

![launch_task_page](./images/launch_task_page.png)

<div align="center"><h2>Please Restart the Kernel</h2></div>

In [49]:
import IPython
app = IPython.Application.instance()
app.kernel.do_shutdown(True)

{'status': 'ok', 'restart': True}